# Reinforcement Learning: Theory and Algorithms

* 第5章における各定理，補題，系などをまとめます．証明は初見でもこれを参考にすれば導出できるように丁寧に記します．
* 必要に応じて実装を行います．

# Agonostic Learning

何らかの仮説クラス$\mathcal{H}$を考えます．このクラスは例えば方策の集合だったり価値関数の集合だったりするかもしれません．ここで，各仮説$f \in \mathcal{H}$には，決定論的方策$\pi_f: \mathcal{S} \rightarrow \mathcal{A}$が対応しているとします．具体的にいくつかのケースを以下のように考えます．

**仮説クラスの例**
* $\mathcal{H}$が方策クラス
* $\mathcal{H}$ が行動価値関数の場合：
	* $f \in \mathcal{H}$に対応する方策は、その価値関数に対してGreedyな方策$\pi_f(s, h)=\operatorname{argmax}_a f_h(s, a)$ となります．
* $\mathcal{H}$ がMDPクラスの場合（$f \in \mathcal{H}$がMDP）である場合：
	* $f \in \mathcal{H}$ に対応する方策は，そのMDPにおける最適方策$\pi_f$となります．

ここで，$\mathcal{H}$から誘導される方策の集合を$\Pi = \{\pi_f \mid f \in \mathcal{H}\}$とします．Agonostic Learningにおける目標は，$\Pi$の中で以下のような期待収益を最大化させる方策を見つけることです．
$$
\max_{\pi \in \Pi} \mathbb{E}_{s_0 \sim \mu} V^\pi(s_0) \quad \quad \text{(0.1)}
$$
## 二値分類と強化学習

Agonostic Learningについて考えるために，二値分類が特殊なケースの強化学習であるということをとらえておきます．つまり，以下のようにできます．

**ホライゾン**
* $H=1$ とします．
**行動空間**
* $|\mathcal{A}|=2$とします．
**初期状態分布**
* $s_0 \sim \mu$とします．
**報酬関数**
* $r(s, a) = \mathbf{1}(\operatorname{label}(s)=a)$ と定義します．$s$の真のラベルが$\operatorname{label}(s)$であり，方策が選んだ行動$a$が真のラベルと一致すれば報酬1，そうでなければ報酬0を得るという構造です．
**学習設定**
* $N$個のiidのラベル付きサンプル$(x_i, y_i)_{i=1}^N$（$x_i \in \mathcal{X}$, $y_i \in \{0,1\}$）が与えられた二値分類問題を考慮
* 分類器の集合$\mathcal{H}$ があり，任意の$h \in \mathcal{H}$ は入力$x$を0か1のラベルに写像する関数$h: \mathcal{X} \rightarrow \{0,1\}$とします．
* $\mathbf{1}(h(x) \neq y)$を設定します．これは$h(x)=y$ なら0、そうでなければ1を取る指示関数です．

二値分類を考えるうえで重要なのは，二値分類は入力の**空間が無限であっても汎化が可能**であるということです．つまり状態空間が無限の場合でも汎化が可能です．では，汎化誤差を追うためにいくつかの定義を行いましょう．

**経験平均誤差**(経験データからの誤差の推定値)
$$
\widehat{\operatorname{err}}(h)=\frac{1}{N} \sum_{i=1}^N \mathbf{1}\left(h\left(x_i\right) \neq y_i\right)
$$
**真の誤差**(真のデータを用いる誤差)
$$
\operatorname{err}(h)=\mathbb{E}_{(X, Y) \sim D} \mathbf{1}(h(X) \neq Y)
$$
ヘフディングの不等式から，以下の誤差推定上界が成り立ちます．
$$
|\operatorname{err}(h)-\widehat{\operatorname{err}}(h)| \leq \sqrt{\frac{1}{2 N} \log \frac{2}{\delta}}
$$
ヘフディングの不等式と，ユニオンバウンドを組み合わせることで，**オッカムのカミソリバウンド**という上界を得ることができます．

### 命題5.1：オッカムのカミソリバウンド

$\mathcal{H}$が有限であるとします．経験平均誤差を最小にする分類器を$\widehat{h}=\arg \min _{h \in \mathcal{H}} \widehat{\text { err }}(h)$ とします．このとき，確率 $1-\delta$ 以上で以下が成り立ちます．

$$
\operatorname{err}(\widehat{h}) \leq \min _{h \in \mathcal{H}} \operatorname{err}(h)+\sqrt{\frac{2}{N} \log \frac{2|\mathcal{H}|}{\delta}}
$$
また，確率 $1-\delta$ 以上で，
$$
\operatorname{err}(\widehat{h}) \leq \min _{h \in \mathcal{H}} \operatorname{err}(h)+\epsilon
$$
を満たすためのサンプル複雑度は，
$$
N \geq \frac{2 \log \frac{2|\mathcal{H}|}{\delta}}{\epsilon^2}
$$
です．

重要な点は，この誤差上界が，入力空間(状態空間)に依存せず，仮説クラスの濃度にのみ依存しているということです．つまり，入力空間が無限であっても上界はそれに関与しません．ただ，これは二値分類の話なので，強化学習とAgonostic Learningの話に戻りましょう．


## 強化学習問題から教師あり学習への帰着

強化学習では方策の意思決定がサンプルデータの偏りをつかさどります．偏りの少ないデータを獲得するためには方策が行動を一様に選択する形が望ましいです．そのような方策をUnif $_{\mathcal{A}}$ と定義します．これを用いて$N$個の軌跡を収集します．次の補題は，Unif $_{\mathcal{A}}$ を用いて，任意の決定論的方策$\pi$の価値関数$V_0^\pi(\mu)$の不変推定量を獲得する条件を示しています．

### 補題5.2：$V_0^\pi(\mu)$の不変推定

* $\tau = (s_0, a_0, r_0, s_1, \ldots)$
$$
V_0^\pi(\mu)=|\mathcal{A}|^H \cdot \mathbb{E}_{\tau \sim \operatorname{Pr}_{\text {Unif }_{\mathcal{A}}}}\left[\mathbf{1}\left(\pi\left(s_0\right)=a_0, \ldots, \pi\left(s_{H-1}\right)=a_{H-1}\right) \sum_{h=0}^{H-1} r\left(s_h, a_h\right)\right]
$$
### 補題5.2の証明

期待収益は以下のようになります．
$$
V_0^\pi(\mu)=\mathbb{E}_{\tau \sim \operatorname{Pr}_\pi}\left[\sum_{h=0}^{H-1} r_h\right]
$$
重点サンプリングの公式から，Unif $_{\mathcal{A}}$のもとでの期待値に変更すると，以下のようになります．
$$
=\mathbb{E}_{\tau \sim \operatorname{Pr}_{\text {Unif }_{\mathcal{A}}}}\left[\frac{\operatorname{Pr}_\pi(\tau)}{\operatorname{Pr}_{\text {Unif }_{\mathcal{A}}(\tau)}} \sum_{h=0}^{H-1} r_h\right]
$$
ここで，期待値の中身の確率は以下のようになります．$\pi$が決定論的なので，
$$
\operatorname{Pr}_\pi(\tau) = \mu(s_0) \prod_{h=0}^{H-1} \mathbf{1}(\pi(s_h)=a_h) P(s_{h+1}|s_h, a_h)
$$
$$
\operatorname{Pr}_{\text {Unif }_{\mathcal{A}}}(\tau) = \mu(s_0) \prod_{h=0}^{H-1} (1/|\mathcal{A}|) P(s_{h+1}|s_h, a_h)
$$
ここで，確率比$\frac{\operatorname{Pr}_\pi(\tau)}{\operatorname{Pr}_{\text {Unif }_{\mathcal{A}}(\tau)}}$はUnif $_{\mathcal{A}}$  が$\pi$と同じ行動系列を取った場合にのみ非ゼロとなりその場合の値は，
$$
(1/(1/|\mathcal{A}|))^H = |\mathcal{A}|^H
$$
です．よって，これを代入すると補題の式が得られます．$|\mathcal{A}|^H$ という係数は推定量が，ホライゾンを経るほど高分散になる可能性が高いことを示唆しています．

実際に補題5.2に従って得られた価値関数の推定量を以下のように示します．
$$
\widehat{V}_0^\pi(\mu)=\frac{|\mathcal{A}|^H}{N} \sum_{n=1}^N \mathbf{1}\left(\pi\left(s_0^n\right)=a_0^n, \ldots \pi\left(s_{H-1}^n\right)=a_{H-1}^n\right) \sum_{t=0}^{H-1} r\left(s_t^n, a_t^n\right)
$$
これを用いて強化学習におけるオッカムのカミソリバウンドを出します．

### 命題5.3：強化学習におけるオッカムのカミソリバウンド

* $\delta > 0$
* $\Pi$：有限の方策集合
* $\widehat{\pi}=\arg \max _{\pi \in \Pi} \widehat{V}_0^\pi(\mu)$ 
このとき，確率$1-\delta$以上で以下が成り立ちます．
$$
V_0^{\widehat{\pi}}(\mu) \geq \max _{\pi \in \Pi} V_0^\pi(\mu)-H|\mathcal{A}|^H \sqrt{\frac{2}{N} \log \frac{2|\Pi|}{\delta}}
$$

### 命題5.3の証明
TODO

ここでは，状態空間が無限の場合でもオッカムのカミソリバウンドはそれに関与しないということがわかりましたが，一方で仮説クラスが無限である場合はどうでしょうか？一応これも対応することができます．VC次元の概念を用いることでバウンドを書くことができます．VC次元については対応するsyumi-noteの要素を参考にしてください．

## Agonostic Learningは指数ホライゾンの依存から抜けられない(下界で示される)
Agonostic Learningは，どんなに良いアルゴリズムを用いても，そのサンプル複雑度の下界に指数ホライゾンが出てきます．それを以下の命題で確認しましょう．

### 命題5.4：生成モデルを用いた下界
* アルゴリズム$\mathcal{A}$ が生成モデルにアクセスできる
* $|\Pi|=|\mathcal{A}|^H$ 
* $\mathcal{A}$が（$\Pi$ に含まれるとは限らない）任意の$\pi$を返し，
$$
V_0^\pi(\mu) \geq \max _{\pi' \in \Pi} V_0^{\pi'}(\mu)-0.5
$$
を確率$1/2$ より大きく満たすならば，$\mathcal{A}$は生成モデルに対して，最低でも以下のサンプルを必要とします．
$$
N \geq c|\mathcal{A}|^H
$$

### 命題5.4の証明
お気持ち理解がむずくてわからん．．．








## Linear Realizability

教師あり学習における線形回帰や二値分類はかなり研究がされています．さらにこれらの収束性についても，特徴量の次元に対して多項式的なサンプル複雑度が得られます．この良い特性をRLで考えられないかというのがこの節でのテーマです．

ここでは，問題設定としてオフライン強化学習の設定を考えます．つまり，
* エージェントはMDPに直接アクセスできない
* 各ステップ$h \in [H]$におけるデータ分布$\mu_h \in \Delta(\mathcal{S}_h \times \mathcal{A})$ から得られたデータが与えられる
* データセットは，$\{D_h\}_{h=0}^{H-1}$ であり、$(s, a, r, s') \in \mathcal{S}_h \times \mathcal{A} \times \mathbb{R} \times \mathcal{S}_{h+1}$ という形式のi.i.d.サンプルから構成される
* $(s, a) \sim \mu_h$
* $r \sim r(s, a)$
* $s' \sim P(s, a)$ 
を考えます．さらに今回は，できるだけ少ないサンプルで方策の価値を近似的に推定するオフライン方策評価をタスクとしてを考えます．

さらに，すべての方策に対して，以下の仮定が成り立つものとします．かなり強い仮定です．

### 仮定5.5：線形実現可能性

全ての$\pi: \mathcal{S} \rightarrow \Delta(\mathcal{A})$ に対して，ある $\theta_0^\pi, \ldots \theta_{H-1}^\pi \in \mathbb{R}^d$ が存在し，全ての $(s, a) \in \mathcal{S} \times \mathcal{A}$ と $h \in [H]$ に対して，
$$
Q_h^\pi(s, a)=\left(\theta_h^\pi\right)^{\top} \phi(s, a)
$$
が成り立ちます．

また，データセットにおける特徴量のカバレッジの仮定もおきます．この仮定は，$\mu$がD-optimal計画であることと等価です．

### 仮定5.6：特徴量のカバレッジ

**仮定の仮定**
* 全ての $(s, a) \in \mathcal{S} \times \mathcal{A}$ に対し，特徴マップは $\|\phi(s, a)\|_2 \leq 1$ と有界

各 $h \in [H]$ に対して、データ分布 $\mu_h$ は以下を満たすと仮定します．
* $I$ は $d \times d$ の単位行列とします．
$$
\mathbb{E}_{(s, a) \sim \mu_h}\left[\phi(s, a) \phi(s, a)^{\top}\right]=\frac{1}{d} I
$$
これらの仮定は，かなり強いものになっており，ホライゾンが1の場合は，$\theta_0^\pi$ をいい感じに推定することができます．一方で，以下の定理にしめすように，ロングホライゾンな設定におけるオフライン方策評価は十分ではありません．

### 定理5.7：ホライゾンとサンプル数

仮定5.6が成り立つとします．方策と特徴マッピングを入力とするアルゴリズムを固定します．仮定5.5を満たすあるMDPが存在し，任意の$\pi: \mathcal{S} \rightarrow \Delta(\mathcal{A})$ に対して，アルゴリズムが$\pi$の価値を定数加法的近似誤差で確率$0.9$以上で出力するためには，
$$
\Omega\left((d / 2)^H\right)
$$ 
のサンプル数を必要とします．